In [6]:
import pandas as pd
link_small = pd.read_csv('movies_data/links_small.csv') # 9125
link_small.dropna(inplace=True)
movies_metadata = pd.read_csv('movies_data/movies_metadata.csv') 
ratings_small = pd.read_csv('movies_data/ratings_small.csv') 


/var/folders/q3/w058rs213y1fbnz4r36x5z2m0000gn/T/ipykernel_59551/272450405.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata = pd.read_csv('movies_data/movies_metadata.csv')


link_small：movieId(=movieId in ratings_small),imdbId (imdb_id in movies_metadata), tmdbId (id in movies_metadata)

In [10]:
movies_metadata.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [15]:
movies_metadata[movies_metadata['imdb_id']=='tt0114709']

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [20]:
ratings_small.groupby('userId').count().max()

movieId      2391
rating       2391
timestamp    2391
dtype: int64

In [21]:
ratings_small

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [11]:
link_small

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9120,162672,3859980,402672.0
9121,163056,4262980,315011.0
9122,163949,2531318,391698.0
9123,164977,27660,137608.0


LSTM input:
- all interaction cuts into 10 internals (t=10), x_t = 这一个internal看过的电影的popularity score

GNN input: 
- urs: random embedding concatenate with LSTM popualrity suscipecity embedding(LSTM final hidden state = influence by popularity)
- item: description vec + (popularity embedding)

----
- user_interaction_history (user_num, sequence_length=10, )
- item_popularity (item_num, 1)
- item_description_vec (item_num, description_dim)

user_iteraction_hidtory:

In [52]:
movies_metadata['id'] = movies_metadata['id'].astype('str')
link_small['tmdbId'] = link_small['tmdbId'].astype(int).astype('str')
movie_info = pd.merge(
    link_small,
    movies_metadata[['id', 'popularity']],
    left_on='tmdbId',
    right_on='id',
    how='left'
)
movie_popularity_dict = dict(zip(
    movie_info['movieId'].astype(int), 
    movie_info['popularity'].astype(float)
)) # map of movieId(in ratings_small) to popularity


In [54]:
import numpy as np
from collections import defaultdict

# 按用户ID和时间戳排序
ratings_small_sorted = ratings_small.sort_values(['userId', 'timestamp'])

# 计算每个用户的交互序列
user_sequences = defaultdict(list)
for _, row in ratings_small_sorted.iterrows():
    user_sequences[row['userId']].append({
        'movieId': row['movieId'],
        'timestamp': row['timestamp'],
        'rating': row['rating']
    })

def split_sequence(sequence, num_intervals=10):
    if len(sequence) == 0:
        return [0] * num_intervals  # 如果没有交互，返回10个0
    
    # 按时间戳排序
    sorted_seq = sorted(sequence, key=lambda x: x['timestamp'])
    # 按照总数量平均分配到各个时间段
    seq_length = len(sorted_seq)
    interval_size = max(1, seq_length // num_intervals)  # 每个区间至少有1个元素
    
    # 初始化时间段的电影列表
    intervals = [[] for _ in range(num_intervals)]
    
    # 将交互按顺序平均分配到时间段
    for i, interaction in enumerate(sorted_seq):
        interval_idx = min(i // interval_size, num_intervals - 1)
        movie_id = interaction['movieId']
        popularity = movie_popularity_dict.get(movie_id, 0)
        intervals[interval_idx].append(popularity)
    
    # 计算每个时间段的平均popularity
    avg_popularity = []
    for interval in intervals:
        if len(interval) > 0:
            avg_popularity.append(np.mean(interval))
        else:
            avg_popularity.append(0)
    
    return avg_popularity

# # 3. 重新计算user_interval_sequences
user_interval_sequences = {
    user_id: split_sequence(seq) 
    for user_id, seq in user_sequences.items()
}


In [55]:
user_interval_sequences

{1.0: [14.780107000000001,
  15.0054685,
  9.29025,
  13.027679499999998,
  10.0566,
  11.927081000000001,
  12.222143500000001,
  8.8342205,
  9.8799675,
  11.516345],
 2.0: [32.34946228571429,
  14.192915000000001,
  11.730826285714286,
  14.276814,
  15.993886142857141,
  12.38450357142857,
  14.44926485714286,
  8.765753142857141,
  7.3284587142857145,
  7.574102846153847],
 3.0: [10.416652,
  9.148622,
  7.4523098,
  52.355382999999996,
  25.229372,
  15.870816999999999,
  17.1992344,
  38.7419646,
  16.134399199999997,
  17.740635333333334],
 4.0: [22.106328400000002,
  10.2753053,
  12.9665494,
  16.881563,
  12.324166,
  7.6837317,
  10.388892899999998,
  9.254189200000003,
  13.902858349999997,
  12.873013208333333],
 5.0: [12.9478502,
  12.951432299999999,
  19.05949,
  14.899134899999998,
  22.1454198,
  14.304296200000001,
  12.667132200000001,
  11.5522521,
  11.030814099999999,
  10.6167535],
 6.0: [11.65189575,
  10.44509175,
  18.783862,
  11.45679075,
  21.39847725,
  

item_populartiy:

In [56]:
movie_popularity_dict

{1: 21.946943,
 2: 17.015539,
 3: 11.7129,
 4: 3.859495,
 5: 8.387519,
 6: 17.924927,
 7: 6.677277,
 8: 2.561161,
 9: 5.23158,
 10: 14.686036,
 11: 6.318445,
 12: 5.430331,
 13: 12.140733,
 14: 5.092,
 15: 7.284477,
 16: 10.137389,
 17: 10.673167,
 18: 9.026586,
 19: 8.205448,
 20: 7.337906,
 21: 12.669608,
 22: 10.701801,
 23: 11.065939,
 24: 12.133094,
 25: 10.332025,
 26: 1.845899,
 27: 8.681325,
 28: 2.228434,
 29: 9.822423,
 30: 1.100915,
 31: 9.481338,
 32: 12.297305,
 34: 14.404764,
 35: 1.493361,
 36: 6.891317,
 37: 0.114469,
 38: 7.777735,
 39: 9.882383,
 40: 0.894647,
 41: 4.561387,
 42: 9.87957,
 43: 10.979269,
 44: 10.870138,
 45: 10.448481,
 46: 4.485958,
 47: 18.45743,
 48: 13.280069,
 49: 3.302524,
 50: 16.302466,
 52: 15.899134,
 53: 1.361286,
 54: 5.982764,
 55: 4.093443,
 57: 8.416802,
 58: 10.168437,
 59: 0.120789,
 60: 10.673296,
 61: 4.733526,
 62: 7.137117,
 63: 11.700115,
 64: 2.174487,
 65: 6.387469,
 66: 2.49535,
 68: 2.518051,
 69: 14.56965,
 70: 15.339153,
 7

item_description_vectors:

In [59]:
# build a map: key: movieId, value: description vector
movie_des = pd.merge(
    link_small[['movieId', 'tmdbId']],
    movies_metadata[['id', 'overview']],
    left_on='tmdbId',
    right_on='id',
    how='left'
)
movie_des.drop(columns=['id'], inplace=True)
movie_des.head(3)

,movieId,tmdbId,overview
0,1,862,"Led by Woody, Andy's toys live happily in his ..."
1,2,8844,When siblings Judy and Peter discover an encha...
2,3,15602,A family wedding reignites the ancient feud be...


In [62]:
# use a pretrained BERT model to encode the overview
import torch
from transformers import BertTokenizerFast, BertModel
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/Users/yaqi/Library/Python/3.9/lib/python/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [67]:
MAX_LENGTH = 256  

def get_bert_embedding(text):
    text = str(text).lower().strip()
    if pd.isna(text) or text == '':
        return np.zeros(768)  # BERT base模型的输出维度是768
        
    inputs = tokenizer(
        text,
        max_length=MAX_LENGTH,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    
    with torch.no_grad():
        outputs = model(**inputs)
        embedding = outputs.last_hidden_state[:, 0, :].numpy()
    
    return embedding.squeeze()



movie_description_dict = {}
for _, row in movie_des.iterrows():
    movie_id = row['movieId']
    overview = row['overview']
    embedding = get_bert_embedding(overview)
    movie_description_dict[movie_id] = embedding

KeyboardInterrupt: 

dataloader

In [78]:
import pickle
# save
with open('data/user_interval_sequences.pkl', 'wb') as f:
    pickle.dump(user_interval_sequences, f)
with open('data/movie_description_dict.pkl', 'wb') as f:
    pickle.dump(movie_description_dict, f)


# load
with open('data/user_interval_sequences.pkl', 'rb') as f:
    user_interval_sequences = pickle.load(f)
with open('data/movie_description_dict.pkl', 'rb') as f:
    movie_description_dict = pickle.load(f)

ratings_small = pd.read_csv('movies_data/ratings_small.csv')
ratings_small.dropna(inplace=True)


In [73]:
import torch
from torch.utils.data import Dataset, DataLoader
import scipy.sparse as sp
import numpy as np

class MovieLensGraphDataset(Dataset):
    def __init__(self, 
                 user_interval_sequences,  # 用户的时序交互历史(popularity)
                 ratings_df,              # 原始评分数据
                 movie_description_dict,  # 电影描述向量
                 min_rating=3.5):         # 定义正样本的最小评分
        
        self.user_interval_sequences = user_interval_sequences
        self.movie_description_dict = movie_description_dict
        
        # 1. 创建用户和物品的ID映射
        self.user_ids = sorted(list(user_interval_sequences.keys()))
        self.movie_ids = sorted(list(movie_description_dict.keys()))
        
        self.user_id_map = {id: i for i, id in enumerate(self.user_ids)}
        self.movie_id_map = {id: i for i, id in enumerate(self.movie_ids)}
        
        # 2. 构建交互图的邻接矩阵
        self.interaction_matrix = self._create_interaction_matrix(
            ratings_df, 
            min_rating
        )
        
        # 3. 预处理所有物品的描述向量
        self.item_embeddings = self._process_item_embeddings()
        
    def _create_interaction_matrix(self, ratings_df, min_rating):
        # 构建稀疏邻接矩阵
        user_indices = []
        item_indices = []
        
        for _, row in ratings_df.iterrows():
            if row['rating'] >= min_rating:  # 只考虑正样本
                if row['userId'] in self.user_id_map and row['movieId'] in self.movie_id_map:
                    user_idx = self.user_id_map[row['userId']]
                    item_idx = self.movie_id_map[row['movieId']]
                    user_indices.append(user_idx)
                    item_indices.append(item_idx)
        
        values = np.ones(len(user_indices))
        
        # 创建稀疏矩阵
        adj_matrix = sp.coo_matrix(
            (values, (user_indices, item_indices)),
            shape=(len(self.user_ids), len(self.movie_ids))
        )
        
        return adj_matrix
    
    def _process_item_embeddings(self):
        # 将所有电影的描述向量整理成张量
        embeddings = []
        for movie_id in self.movie_ids:
            vec = self.movie_description_dict.get(movie_id, np.zeros(768))
            embeddings.append(vec)
        return torch.FloatTensor(embeddings)
    
    def __len__(self):
        return len(self.user_ids)
    
    def __getitem__(self, idx):
        user_id = self.user_ids[idx]
        
        # 1. 获取用户的交互历史序列
        interaction_history = torch.FloatTensor(
            self.user_interval_sequences[user_id]
        )
        
        # 2. 获取该用户的交互图信息
        user_idx = self.user_id_map[user_id]
        user_interactions = self.interaction_matrix.getrow(user_idx).toarray().squeeze()
        interacted_items = np.where(user_interactions > 0)[0]
        
        return {
            'user_id': user_idx,
            'interaction_history': interaction_history,  # shape: [10]
            'interacted_items': torch.LongTensor(interacted_items),
            'interaction_graph': torch.FloatTensor(user_interactions)
        }
    
    def get_item_embeddings(self):
        return self.item_embeddings

def collate_fn(batch):
    # 自定义批处理函数
    batch_interaction_history = torch.stack([item['interaction_history'] for item in batch])
    
    # 收集所有批次的用户ID
    batch_user_ids = torch.tensor([item['user_id'] for item in batch])
    
    # 收集交互图信息
    batch_interaction_graphs = torch.stack([item['interaction_graph'] for item in batch])
    
    return {
        'user_ids': batch_user_ids,
        'interaction_history': batch_interaction_history,  # 用于LSTM
        'interaction_graphs': batch_interaction_graphs,    # 用于GNN
    }

# 创建DataLoader
def create_dataloader(user_interval_sequences, ratings_df, movie_description_dict, 
                     batch_size=32, shuffle=True):
    dataset = MovieLensGraphDataset(
        user_interval_sequences,
        ratings_df,
        movie_description_dict
    )
    
    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        collate_fn=collate_fn
    ), dataset



In [74]:
# 测试代码
dataloader, dataset = create_dataloader(
    user_interval_sequences,
    ratings_small,
    movie_description_dict
)

# 打印一个batch的数据结构
for batch in dataloader:
    print("Batch structure:")
    print("User IDs shape:", batch['user_ids'].shape)
    print("Interaction history shape:", batch['interaction_history'].shape)
    print("Interaction graphs shape:", batch['interaction_graphs'].shape)
    print("\nItem embeddings shape:", dataset.get_item_embeddings().shape)
    break

Batch structure:
User IDs shape: torch.Size([32])
Interaction history shape: torch.Size([32, 10])
Interaction graphs shape: torch.Size([32, 4856])

Item embeddings shape: torch.Size([4856, 768])
